{{badge}}

https://medium.com/analytics-vidhya/introduction-to-automatic-hyperparameter-optimization-with-hyperopt-e0b9c84d1059<br>
https://medium.com/district-data-labs/parameter-tuning-with-hyperopt-faa86acdfdce
<br>
https://github.com/hyperopt/hyperopt/issues/267

#Extra code for trials in hyperopt



```
search_space = {'alpha':hp.choice('alpha',[10**x for x in range(-1,2)])
                ,'penalty':hp.choice('penalty',['l1']),
                'loss':hp.choice('loss',['hinge'])
                }
#from hyperopt import Trials
spark_trials = SparkTrials(parallelism=2)


best = fmin(fn=opt_alpha_sgd_para, 
            space=search_space, 
            algo=tpe.suggest, 
            max_evals=16, 
            trials = spark_trials
           #,return_argmin=False
            )
```





```
def trim_axs(axs,N):
  axs = axs.flat
  for ax in axs[N:]:
    ax.remove()
  return axs[:N]
```





```
def plot_parameters(search_space,trials,dist_plots=True,**kwargs):
  #can pass dist_plot=True
  #if kwargs is defined, we can pass arguements that are not defined in function definition
  #in a way kwargs stores the passed undefined paramers
  #below we are making a final list of total parameters.

  #params hold keys of our search_space
  #ncols is used for no. of subplots in a row.
  defaults = {'params' : search_space.keys(), 'ncols':3, 'constrained_layout' :True}

  #checking whether keys of our search_space are passed as extra arguments and
  # if defined then we'll that key from 'defaults' as to avoid dups of params as we are combining all the params in next step
  defaults = {k:v for k,v in defaults.items() if k not in kwargs}

  #storing all the unique params in kwargs
  kwargs = {**kwargs, **defaults}

  #storing keys of search_space in params
  params = kwargs['params']

  #now after below kwargs hold only those params except keys of search space
  del kwargs['params']
  
  results_df = pd.DataFrame(columns=['tid', *params, 'status'])


  for t in trials.trials:
    #vals - hold positions of passed search_space params
    #and positions as a list(ex: 'alpha':[0])
    result_dict =t['misc']['vals']
    #changing position list value as interger and assigning to same key
    result_dict = {d:v[0] for d,v in result_dict.items()}

    #getting the respective values of above positions dict
    #result_dict holds the alpha,corr loss values
    result_dict = space_eval(search_space,result_dict)
    result_dict['tid'] = t['tid']
    result_dict.update(t['result'])

    results_df = pd.concat([results_df, pd.DataFrame(result_dict, index=[0])], axis=0, ignore_index = True)
    
    #storing only those entries that have status 'ok
  results_df = results_df[results_df.status=='ok']

    #changing datatypes of columns which are having as 'object' to respectives
  results_df = results_df.infer_objects()
    
#from here customize the plot according to your wish

    #to know in how many rows we can fit our plots.
  nrows = int(math.ceil((len(params)+1)/kwargs['ncols']))

  if('figsize' not in kwargs.keys()):
    kwargs['figsize'] = (15, 5*nrows)
    
  f,ax = plt.subplots(nrows =nrows, **kwargs)
  ax = trim_axs(axs, len(params)+1)
  f.suptitle('Parameter space tried by Hyperopt & Loss')

  for ax, p in zip(ax, [*params,'loss']):
    if p=='loss':
      sns.scatterplot(x=results_df.tid, y=results_df.loc[:p], alpha =0.8, ax=ax)
      as.set_title('Scatterplot of loss vs trial numer')
    elif dist_plots:
      if pd.api.types.is_bool_dtype(results_df.loc[:p]) or pd.api.types.is_string_dtype(results_df.loc[:p]):
        sns.countplot(results_df.loc[:,p], ax=ax)
      else:
        sns.distplot(results_df.loc[:,p].astype('float'), bins=10, ax=ax)
      ax.set_title(p)
    else:
      sns.scatterplot(x=resulsts_df.loc[:, p], y=results_df.loss, alpha=0.8, ax=ax)
      ax.set_title(f'{p} vs loss')
  
  return results_df
```

